In [49]:
# Import the Earth Engine Python Package
import ee
from IPython.display import Image
import pprint
import ast
import pandas as pd

# Configure the pretty printing output.
pp = pprint.PrettyPrinter(depth=4)

# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()

In [50]:
taihu = pd.read_csv('./data/chla_taihu.csv')
taihu['record_date'] = pd.to_datetime(taihu.record_date)
sorted_datetime = taihu.record_date.sort_values()
sorted_datetime

107   2011-08-11
218   2011-08-11
48    2011-08-11
268   2011-08-11
162   2011-08-11
11    2011-08-11
243   2011-08-11
376   2011-08-11
294   2011-08-11
129   2011-08-11
78    2011-08-11
219   2011-08-18
270   2011-08-18
382   2011-08-18
235   2011-08-18
113   2011-08-18
171   2011-08-18
8     2011-08-18
293   2011-08-18
139   2011-08-18
76    2011-08-18
52    2011-08-18
166   2011-12-13
70    2011-12-13
89    2011-12-13
204   2011-12-13
43    2011-12-13
109   2011-12-13
146   2011-12-13
377   2011-12-13
         ...    
340   2013-11-22
352   2013-11-22
335   2013-11-22
189   2014-02-22
392   2014-02-22
4     2014-02-22
297   2014-02-22
281   2014-02-22
306   2014-02-22
346   2014-02-22
354   2014-02-22
260   2014-02-22
33    2014-02-22
313   2014-02-22
133   2014-02-22
157   2014-02-22
57    2014-02-22
232   2014-02-22
344   2014-02-22
325   2014-02-22
328   2014-02-22
122   2014-02-22
153   2014-02-22
203   2014-02-22
172   2014-02-22
338   2014-02-22
96    2014-02-22
366   2014-02-

In [51]:
taihu.head()

,record_date,station,chla,coordinates
0,2013-05-13,0,1.5,"[120.19067,31.51317]"
1,2013-06-17,0,1.5,"[120.19067,31.51317]"
2,2012-05-14,0,2.4,"[120.19067,31.51317]"
3,2012-11-18,0,2.4,"[120.19067,31.51317]"
4,2014-02-22,0,2.8,"[120.19067,31.51317]"


In [52]:
sorted_datetime[0] # how to change to 'yyyy-mm-dd'

Timestamp('2013-05-13 00:00:00')

In [53]:
str(sorted_datetime[0]).strip( ' 00:00:00')

'2013-05-13'

In [54]:
# remove duplicate station coordinates
unique_coordinates = taihu.coordinates.drop_duplicates()
coordinate_count = unique_coordinates.count()
unique_coordinates.index

Int64Index([  0,  21,  41,  64,  87, 106, 127, 149, 156, 162, 184, 202, 225,
            248, 265, 283, 301, 308, 315, 322, 328, 334, 340, 346, 352, 358,
            373],
           dtype='int64')

In [55]:
type(ast.literal_eval(taihu.coordinates[0]))

list

In [56]:
a = '20k'
b=a.strip('k')
b

'20'

In [57]:
sample_coordinates = []

for i in unique_coordinates.index:
    cooordinates = ast.literal_eval(unique_coordinates[i])
    sample_coordinates.append(cooordinates)

pp.pprint(sample_coordinates)

[[120.19067, 31.51317],
 [120.19433, 31.47633],
 [120.18796, 31.43609],
 [120.18733, 31.41117],
 [120.18017, 31.33833],
 [120.17062, 31.24816],
 [119.945, 31.3145],
 [120.11866, 30.96367],
 [120.45383, 31.02167],
 [120.2955, 31.3865],
 [120.37691, 31.43511],
 [120.02817, 31.45001],
 [120.03182, 31.39761],
 [120.05612, 31.3081],
 [120.02333, 31.19055],
 [119.96731, 31.10789],
 [120.143856, 31.116508],
 [120.18982, 30.99104],
 [120.23271, 31.01261],
 [120.37909, 30.98091],
 [120.51329, 31.08941],
 [120.33567, 31.09903],
 [120.40596, 31.17683],
 [120.464785, 31.20565],
 [120.33149, 31.24482],
 [120.241535, 31.353396],
 [120.14594, 31.4068]]


In [62]:
# Create a feature collection of all sample points

sample_points = []

for i in range(0,len(sample_coordinates)):
    point = ee.Geometry.Point(sample_coordinates[i])
    sample_points.append(point)
    
all_sample_points = ee.FeatureCollection(sample_points)
print(all_sample_points)

ee.FeatureCollection({
  "type": "Invocation",
  "arguments": {
    "features": [
      {
        "type": "Invocation",
        "arguments": {
          "geometry": {
            "type": "Point",
            "coordinates": [
              120.19067,
              31.51317
            ]
          }
        },
        "functionName": "Feature"
      },
      {
        "type": "Invocation",
        "arguments": {
          "geometry": {
            "type": "Point",
            "coordinates": [
              120.19433,
              31.47633
            ]
          }
        },
        "functionName": "Feature"
      },
      {
        "type": "Invocation",
        "arguments": {
          "geometry": {
            "type": "Point",
            "coordinates": [
              120.18796,
              31.43609
            ]
          }
        },
        "functionName": "Feature"
      },
      {
        "type": "Invocation",
        "arguments": {
          "geometry": {
            "type": 

In [59]:
# Generate a tile layer url
def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

# Generate a feature collection layer url
def GetFeatureLayerUrl(ee_featurecollection_object):
    map_id = ee.FeatureCollection(ee_featurecollection_object).getMapId()
    featurecollection_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return featurecollection_url_template.format(**map_id)

In [60]:
import ipyleaflet 
taihu_map = ipyleaflet.Map(center=(31.25, 120.3), zoom=10, layout={'height':'600px'})
dc = ipyleaflet.DrawControl()
taihu_map.add_control(dc)
taihu_map

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[31.25, 120.3], controls=(DrawControl(layer=FeatureGroup(), polygon={'shapeOptions': {}}, polyline={'shapeOptions': {}}),), layers=(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size']),), layout=Layout(height='600px'), options=['attribution_control', 'basemap', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], zoom=10)

In [61]:
url = GetFeatureLayerUrl(all_sample_points)

print(url)

taihu_map.add_layer(ipyleaflet.TileLayer(url=url))

https://earthengine.googleapis.com/map/f9e0ac48b9172ce5e2a8aeaaa42782fc/{z}/{x}/{y}?token=f606d44134ce63c1974b30f89797704e


In [14]:
# remove duplicate image dates 
unique_dates = sorted_datetime.drop_duplicates()
count_dates = unique_dates.count()
unique_dates.index


Int64Index([107, 219, 166,  60, 343, 242, 125, 143,  54, 106, 152, 378, 266,
            329, 135, 257,   9, 307, 258, 282,  64, 361,  90, 178, 360, 349,
            189],
           dtype='int64')

In [15]:
sample_dates = []

for i in unique_dates.index:
    dates = str(sorted_datetime[i]).strip( ' 00:00:00')
    sample_dates.append(dates)

pp.pprint(sample_dates)




['2011-08-11',
 '2011-08-18',
 '2011-12-13',
 '2012-01-13',
 '2012-02-13',
 '2012-02-14',
 '2012-03-14',
 '2012-05-14',
 '2012-06-14',
 '2012-07-23',
 '2012-08-15',
 '2012-09-13',
 '2012-10-19',
 '2012-11-17',
 '2012-11-18',
 '2012-12-19',
 '2013-01-15',
 '2013-02-27',
 '2013-02-28',
 '2013-03-16',
 '2013-04-15',
 '2013-05-13',
 '2013-06-17',
 '2013-08-22',
 '2013-11-21',
 '2013-11-22',
 '2014-02-22']


In [22]:
def strip_time(timestamp):
    return str(timestamp).strip( ' 00:00:00')

In [24]:
strip_time(taihu.record_date[0])

'2013-05-13'

In [26]:
taihu['clean_date']=taihu.record_date.apply(strip_time)

In [27]:
taihu.head()

,record_date,station,chla,coordinates,image_start_date,image_end_date,clean_date
0,2013-05-13,1,1.5,"[120.19067,31.51317]",2013-05-06,2013-05-20,2013-05-13
1,2013-06-17,1,1.5,"[120.19067,31.51317]",2013-06-10,2013-06-24,2013-06-17
2,2012-05-14,1,2.4,"[120.19067,31.51317]",2012-05-07,2012-05-21,2012-05-14
3,2012-11-18,1,2.4,"[120.19067,31.51317]",2012-11-11,2012-11-25,2012-11-18
4,2014-02-22,1,2.8,"[120.19067,31.51317]",2014-02-15,2014-03-01,2014-02-22


In [17]:
from datetime import timedelta
offset = timedelta(days=7)

In [19]:
taihu['image_start_date'] = taihu.record_date - offset
taihu['image_end_date'] = taihu.record_date + offset

In [21]:
taihu.head()

,record_date,station,chla,coordinates,image_start_date,image_end_date
0,2013-05-13,1,1.5,"[120.19067,31.51317]",2013-05-06,2013-05-20
1,2013-06-17,1,1.5,"[120.19067,31.51317]",2013-06-10,2013-06-24
2,2012-05-14,1,2.4,"[120.19067,31.51317]",2012-05-07,2012-05-21
3,2012-11-18,1,2.4,"[120.19067,31.51317]",2012-11-11,2012-11-25
4,2014-02-22,1,2.8,"[120.19067,31.51317]",2014-02-15,2014-03-01


In [ ]:
# remove duplicate image dates 
unique_start_dates = taihu.image_start_date.drop_duplicates()
count_dates = unique_start_dates.count()
unique_start_dates.index

image_start_date = []

for i in unique_start_dates.index:
    dates = str(taihu.image_start_date[i]).strip( ' 00:00:00')
    sample_dates.append(dates)

pp.pprint(sample_dates)

In [ ]:
landsat = []   

for i in range(0,len(imageDates)):
    landsatImage = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR").filterDate(image_start_date[i],image_end_date[i])
    landsatTOA.append(landsatImage)